In [95]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Lasso


data_path = './data/'
df_train = pd.read_csv(data_path + 'train_data.csv')
df_test = pd.read_csv(data_path + 'test_features.csv')

# poi : 直接轉TRUE 1 FALSE 0 -- TRUE FALSE 經過READ_CSV會被轉換成Boolean
#train_Y = df_train['poi']
train_Y = df_train['poi'].replace({True: 1, False: 0})
ids = df_test['name']
df_train_corr = df_train  #檢查相關係數用的TRAIN DATA
df_train = df_train.drop(['name', 'poi'] , axis=1)
df_test = df_test.drop(['name'] , axis=1)
test_X = df_test
#df = df_train
df = pd.concat([df_train,df_test])
df.head()

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,NaN,-3504386.0,NaN,ken.rice@enron.com,19794175.0,46950.0,18.0,42.0,4.0,NaN,1617011.0,174839.0,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,30.0,NaN,1920000.0,22122.0,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,200000.0,NaN,-4167.0,NaN,rex.shelby@enron.com,1624396.0,22884.0,39.0,13.0,14.0,NaN,NaN,1573324.0,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,800000.0,NaN,NaN,NaN,michael.kopper@enron.com,NaN,118134.0,NaN,NaN,NaN,NaN,602671.0,907502.0,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,1250000.0,NaN,-262500.0,NaN,christopher.calger@enron.com,NaN,35818.0,144.0,199.0,25.0,NaN,375304.0,486.0,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0


In [96]:
print(train_Y.shape[0])

113


In [97]:
# 檢查 DataFrame 空缺值的狀態
def na_check(df_data):
    data_na = (df_data.isnull().sum() / len(df_data)) * 100
    data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Missing Ratio' :data_na})
    display(missing_data.head(20))
na_check(df)

,Missing Ratio
loan_advances,97.260274
director_fees,88.356164
restricted_stock_deferred,87.671233
deferral_payments,73.287671
deferred_income,66.438356
long_term_incentive,54.794521
bonus,43.835616
from_messages,41.095890
from_poi_to_this_person,41.095890
from_this_person_to_poi,41.095890


In [100]:

#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]
test_num = test_X.shape[0]
df=df.fillna(df.mean())
#df = df.fillna(-1)
MMEncoder = MinMaxScaler()
STDEncoder = StandardScaler()
df.head()


19 Numeric Features : ['bonus', 'deferral_payments', 'deferred_income', 'director_fees', 'exercised_stock_options', 'expenses', 'from_messages', 'from_poi_to_this_person', 'from_this_person_to_poi', 'loan_advances', 'long_term_incentive', 'other', 'restricted_stock', 'restricted_stock_deferred', 'salary', 'shared_receipt_with_poi', 'to_messages', 'total_payments', 'total_stock_value']



,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,1.642674e+06,-3.504386e+06,166804.882353,1.979418e+07,46950.0,18.000000,42.000000,4.000000,41962500.0,1.617011e+06,174839.0,2748364.0,166410.555556,420636.0,864.000000,905.000000,505050.0,22542539.0
1,5600000.0,1.642674e+06,-1.140475e+06,166804.882353,1.925000e+07,29336.0,108.000000,88.000000,30.000000,41962500.0,1.920000e+06,22122.0,6843672.0,166410.555556,1111258.0,2042.000000,3627.000000,8682716.0,26093672.0
2,200000.0,1.642674e+06,-4.167000e+03,166804.882353,1.624396e+06,22884.0,39.000000,13.000000,14.000000,41962500.0,1.470361e+06,1573324.0,869220.0,166410.555556,211844.0,91.000000,225.000000,2003885.0,2493616.0
3,800000.0,1.642674e+06,-1.140475e+06,166804.882353,5.987054e+06,118134.0,608.790698,64.895349,41.232558,41962500.0,6.026710e+05,907502.0,985032.0,166410.555556,224305.0,1176.465116,2073.860465,2652612.0,985032.0
4,1250000.0,1.642674e+06,-2.625000e+05,166804.882353,5.987054e+06,35818.0,144.000000,199.000000,25.000000,41962500.0,3.753040e+05,486.0,126027.0,166410.555556,240189.0,2188.000000,2598.000000,1639297.0,126027.0


In [101]:
## 跟LASSO擇一處理 ###檢查共線性，刪除特徵值 參考D15

# 觀察相關係數,列出目標 (TARGET) 與所有欄位之間相關係數，數值最大以及最小各 15 個
df_corr = df_train_corr.corr()
#print(df_corr.to_xarray()) 
print(df_corr['poi'].sort_values())

# max_corr = df_corr[1:12]
# min_corr = df_corr[-10:]
#print(max_corr)
##print(min_corr)

# 參考D30 篩選相關係數1

corr_features = list(df_corr[(df_corr['poi']>0.1) | (df_corr['poi']<-0.1)].index)
corr_features.remove('poi')
print(len(corr_features))
print(corr_features)
corr_features.pop(0)
print(type(corr_features))


## df = df[corr_features]
## train_X = MMEncoder.fit_transform(df[corr_features])
## cross_val_score(estimator, train_X, train_Y, cv=5).mean()


deferral_payments           -0.251548
deferred_income             -0.224654
from_messages               -0.126760
from_this_person_to_poi     -0.096864
to_messages                 -0.023047
shared_receipt_with_poi      0.120467
from_poi_to_this_person      0.127932
expenses                     0.136386
other                        0.225619
long_term_incentive          0.227805
bonus                        0.242170
restricted_stock             0.261102
total_payments               0.271335
salary                       0.371917
total_stock_value            0.443098
exercised_stock_options      0.591972
poi                          1.000000
loan_advances                1.000000
director_fees                     NaN
restricted_stock_deferred         NaN
Name: poi, dtype: float64
15
['bonus', 'deferral_payments', 'deferred_income', 'exercised_stock_options', 'expenses', 'from_messages', 'from_poi_to_this_person', 'loan_advances', 'long_term_incentive', 'other', 'restricted_stock', 'salary',

In [102]:
#參考D12
# 將資料標準化
# 將前述轉換完畢資料 df , 重新切成 train_X, test_X
#train_X = STDEncoder.fit_transform(df[:train_num])
#test_X =  STDEncoder.fit_transform(df[:test_num])
# 將資料最大最小化
## train_X =  MMEncoder.fit_transform(df[:train_num])
## test_X =  MMEncoder.fit_transform(df[:test_num])
train_X=df[:train_num]
test_X=df[:test_num]
train_X =  MMEncoder.fit_transform(train_X[corr_features])
test_X =  MMEncoder.fit_transform(test_X[corr_features])
print(type(corr_features))
print(test_X.shape)
print(train_X.shape)
print(train_Y.shape)

<class 'list'>
(33, 14)
(113, 14)
(113,)


In [71]:
# 原始特徵 + 邏輯斯迴歸
#train_X = MMEncoder.fit_transform(df)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.8853754940711462

In [115]:
## 跟CORR 使用LASSO 檢查共線性，刪除特徵值 參考D40
from sklearn.linear_model import Lasso
# 切分訓練集/測試集
##x_train, x_test, y_train, y_test = train_test_split(diabetes.data, diabetes.target, test_size=0.2, random_state=4)

# 建立一個線性回歸模型
lasso = Lasso(alpha=0.001)

# 將訓練資料丟進去模型訓練
lasso.fit(train_X, train_Y)

# 將測試資料丟進模型得到預測結果
y_pred = lasso.predict(test_X)

# 檢查相關性
lasso.coef_

#
from itertools import compress
L1_mask = list((lasso.coef_>0) | (lasso.coef_<0))
L1_list = list(compress(list(df), list(L1_mask)))
L1_list.pop(0)
print(type(L1_list),L1_list)
train_X=df[:train_num]
test_X=df[:test_num]
train_X =  MMEncoder.fit_transform(train_X[L1_list])
test_X =  MMEncoder.fit_transform(test_X[L1_list])
print(type(corr_features))
print(test_X.shape)
print(train_X.shape)
print(train_Y.shape)

<class 'list'> ['deferred_income', 'exercised_stock_options', 'expenses', 'from_this_person_to_poi', 'long_term_incentive', 'other', 'restricted_stock']
<class 'list'>
(33, 7)
(113, 7)
(113,)


In [116]:
## 模型參數最佳化  D47
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

# 設定要訓練的超參數組合
n_estimators = [50, 100, 200, 300]
max_depth = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

clf = GradientBoostingRegressor(random_state=7)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(train_X, train_Y)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

grid_result.best_params_

# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingRegressor(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Best Accuracy: -0.141219 using {'max_depth': 1, 'n_estimators': 50}


[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    0.3s finished


In [123]:

# 使用三種模型 : 邏輯斯迴歸 / 梯度提升機 / 隨機森林, 參數使用 Random Search 尋找
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
lr = LogisticRegression()
gdbt = GradientBoostingClassifier()
rf = RandomForestClassifier(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

In [118]:
print(df.shape)
print(train_num)
print(train_X.shape)
print(test_X.shape)

(146, 19)
113
(113, 7)
(33, 7)


In [124]:
# 線性迴歸預測檔 (結果有部分隨機, 請以 Kaggle 計算的得分為準, 以下模型同理)
lr.fit(train_X, train_Y)
prediction = lr.predict_proba(test_X)[:,1]
print(lr.coef_)
print(lr.intercept_)
print(prediction)

[[-0.81491946  1.66161839 -0.49875799 -0.47237265 -0.08375361  0.14866116
   0.421834  ]]
[-1.39494207]
[0.35657434 0.25890166 0.09573667 0.09998584 0.11207708 0.30312306
 0.14327118 0.11972444 0.39593711 0.08100515 0.08148157 0.08425456
 0.12927128 0.08022495 0.16387931 0.09426769 0.07348622 0.08123079
 0.13621477 0.10493371 0.12981673 0.14039051 0.12870745 0.08804048
 0.08656915 0.08265899 0.07873464 0.09087519 0.14875436 0.06554486
 0.08108466 0.11681859 0.11839466]


In [130]:
# 線性迴歸預測檔 (結果有部分隨機, 請以 Kaggle 計算的得分為準, 以下模型同理)
lr.fit(train_X, train_Y)
lr_pred = lr.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name': ids, 'poi': lr_pred})
sub['poi'] = sub['poi'].map(lambda x:1 if x>0.3 else 0) 
sub.to_csv('marathon_lr.csv', index=False) 

In [ ]:
#### ** not finish **  檢查AUC

acc = metrics.accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)
print("Feature importance: ", clf_bestparam.feature_importances_)

In [131]:
# 梯度提升機預測檔 
gdbt.fit(train_X, train_Y)
gdbt_pred = gdbt.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name': ids, 'poi': gdbt_pred})
sub['poi'] = sub['poi'].map(lambda x:1 if x>0.5 else 0) 
sub.to_csv('marathon_gdbt.csv', index=False)

In [132]:
# 隨機森林預測檔
rf.fit(train_X, train_Y)
rf_pred = rf.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name': ids, 'poi': rf_pred})
sub['poi'] = sub['poi'].map(lambda x:1 if x>0.3 else 0) 
sub.to_csv('marathon_rf.csv', index=False)

In [135]:
from mlxtend.classifier import StackingClassifier

meta_estimator = GradientBoostingClassifier(tol=100, subsample=0.70, n_estimators=50, 
                                           max_features='sqrt', max_depth=4, learning_rate=0.3)
"""
Your Code Here
"""
stacking = StackingClassifier(classifiers=[lr,gdbt,rf],meta_classifier=meta_estimator)

In [136]:
stacking.fit(train_X, train_Y)
stacking_pred = stacking.predict(test_X)
sub = pd.DataFrame({'name': ids, 'poi': stacking_pred})
sub.to_csv('marathon_stacking1.csv', index=False)